In [ ]:
%pip install semantic-link-labs

In [ ]:
import sempy_labs
import sempy.fabric as fabric  
from sempy_labs._helper_functions import resolve_workspace_name_and_id  
  
# Get both name and ID of current workspace  
workspace_name, workspace_id = resolve_workspace_name_and_id()
# List all connections  
connections_df = sempy_labs.list_connections()
#display(connections_df)

#### Check connections to help verify which connections can be reused

In [ ]:
# Define the list of connection types to keep
connection_types_to_keep = ["AzureDatabricksWorkspace", "AzureDataLakeStorage"]

# Filter the DataFrame for the desired connection types
filtered_connections_df = connections_df[connections_df["Connection Type"].isin(connection_types_to_keep)]

# Display the filtered DataFrame
display(filtered_connections_df)

#### Check for existing Databricks mirrored items

In [ ]:
item_list_df = fabric.list_items(workspace= workspace_id)
#display(item_list_df)

In [ ]:
# Define the list of connection types to keep when checking for already used names
items_types_to_keep = ["MirroredAzureDatabricksCatalog"]

# Filter the DataFrame for the desired connection types
item_list_filtered = item_list_df[item_list_df["Type"].isin(items_types_to_keep)]

# Display the filtered DataFrame - items created by UI will have empty description, ones created by automation with say "Auto-created"
display(item_list_filtered)

In [ ]:

item_name_nameonly = item_list_filtered["Display Name"]
#display(item_name_nameonly)

#### Get Databricks mirror item definition

In [ ]:
from sempy_labs._helper_functions import get_item_definition  

def get_databricks_mirror_definition(item_name, workspace_name):  
    
    definition = get_item_definition(  
        item=item_name,  
        type="MirroredAzureDatabricksCatalog", 
        workspace= workspace_name,  
        return_dataframe=False,  
        decode=True  
    )
    print(f'###################### {item_name} ########################')
    print(definition)


In [ ]:
# Print out item definitions
for item_name in item_name_nameonly:
    get_databricks_mirror_definition(item_name, workspace_name)

#### What can go wrong?


###### Azure Key Vault

If Firewall is set to Public Access Disabled, you will get an error:
- _IOException: 403 {"error":{"code":"Forbidden","message":"Connection is not an approved private link and caller was ignored because bypass is not set to 'AzureServices' and PublicNetworkAccess is set to 'Disabled'_
- _IOException: 403 {"error":{"code":"Forbidden","message":"Public network access is disabled and request is not from a trusted service nor via an approved private link._

To resolve this
- add your Workspace identity as a Secrets User or Secrets Reader to the AKV and 
- add a Managed Private Endpoint scoped to the relevant key vault to your workspace and and approve it. Be patient, it takes a bit to sort itself out.





In [ ]:
fabric_ws_akv = "https://fab279akv1.vault.azure.net/"
adb_ws_id_s = "databricks-ws-id"
adb_ws_token_s= "databricks-ws-token"

# get the Databricks workspace id and PAT
dbx_workspace = notebookutils.credentials.getSecret(
    fabric_ws_akv,
    adb_ws_id_s)
 
 # will print as [REDACTED], just testing it works
print(f'Databricks workspace name is redacted: {dbx_workspace}')